<a href="https://colab.research.google.com/github/Brookyne/Yolo_learning/blob/main/YOLO_Drone_Video_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO — Chạy mô hình nhận diện Drone trên **video** (Colab-ready)

Notebook này đã cấu hình sẵn để bạn chạy mô hình YOLO (Ultralytics) trên **file video**.

👉 Bạn chỉ cần **điền đường dẫn** `MODEL_PATH` (tới `best.pt`) và `VIDEO_PATH` (tới video đầu vào). Có 2 cách chạy:

1. **Cách 1 — Ultralytics.predict** (nhanh & đơn giản)
2. **Cách 2 — OpenCV xử lý từng khung hình** (tuỳ biến cao, dễ chèn logic riêng)

---
**Gợi ý:** Nếu model/video nằm trong Google Drive, mount Drive và trỏ đường dẫn đúng. Nếu không có GPU, code tự chạy CPU.


In [ ]:
#@title 1) Cài thư viện cần thiết (Ultralytics, OpenCV)
!pip -q install ultralytics opencv-python
from ultralytics import YOLO
import cv2, os, glob, time
from IPython.display import HTML
from base64 import b64encode
print("✅ Installed & imported.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Installed & imported.


In [ ]:
#@title 2) (Tuỳ chọn) Mount Google Drive nếu model/video ở Drive
use_drive = False  #@param {type:"boolean"}
if use_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Drive mounted at /content/drive")
else:
    print("ℹ️ Bỏ qua mount Google Drive")

ℹ️ Bỏ qua mount Google Drive


In [ ]:
#@title 3) Thiết lập đường dẫn model & video
MODEL_PATH = "/content/best.pt"
VIDEO_PATH = "https://drive.google.com/file/d/1JA5bb3-8B7oGhGF4uzKFL8QyRSaozVip/view?usp=drive_link" #@param {type:"string"}
OUTPUT_DIR = "/content/runs/detect"  # Thư mục YOLO mặc định

device_prefer = 0  # 0: GPU đầu tiên nếu có, nếu không sẽ tự fallback CPU; có thể đặt 'cpu' để ép CPU
conf_thres = 0.25  #@param {type:"number"}
iou_thres  = 0.45  #@param {type:"number"}
img_size   = 640   #@param {type:"integer"}

print("MODEL_PATH:", MODEL_PATH)
print("VIDEO_PATH:", VIDEO_PATH)

MODEL_PATH: best.pt
VIDEO_PATH: https://drive.google.com/file/d/1JA5bb3-8B7oGhGF4uzKFL8QyRSaozVip/view?usp=drive_link


## Cách 1 — Dùng `model.predict` (nhanh & gọn)

In [ ]:
#@title 4) Chạy predict trực tiếp (Ultralytics)
assert os.path.exists(MODEL_PATH), f"Không tìm thấy model: {MODEL_PATH}"
assert os.path.exists(VIDEO_PATH), f"Không tìm thấy video: {VIDEO_PATH}"

model = YOLO(MODEL_PATH)
results = model.predict(
    source=VIDEO_PATH,
    conf=conf_thres,
    iou=iou_thres,
    imgsz=img_size,
    device=device_prefer,
    save=True,
    save_txt=False,
)

out_dirs = sorted(glob.glob("runs/detect/predict*"))
assert len(out_dirs) > 0, "Không tìm thấy thư mục kết quả trong runs/detect/"
pred_dir = out_dirs[-1]
print("📁 Thư mục kết quả:", pred_dir)

# Tìm video đầu ra (.mp4)
mp4_files = glob.glob(os.path.join(pred_dir, "*.mp4"))
if len(mp4_files) == 0:
    # một số phiên bản đặt tên result khác -> quét sâu thêm
    mp4_files = glob.glob(os.path.join(pred_dir, "**/*.mp4"), recursive=True)
assert len(mp4_files) > 0, "Không tìm thấy file .mp4 đầu ra!"
display_path = mp4_files[0]
print("🎬 Video kết quả:", display_path)

# Hiển thị ngay trong notebook
mp4 = open(display_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=800 controls>
      <source src='{data_url}' type='video/mp4'>
</video>
""")

AssertionError: Không tìm thấy model: best.pt

## Cách 2 — Dùng OpenCV đọc khung hình & vẽ bbox thủ công (tuỳ biến)

In [ ]:
#@title 5) Suy luận khung-hình (OpenCV loop) và xuất video
OUTPUT_PATH = "/content/out_detect.mp4"  #@param {type:"string"}

assert os.path.exists(MODEL_PATH), f"Không tìm thấy model: {MODEL_PATH}"
assert os.path.exists(VIDEO_PATH), f"Không tìm thấy video: {VIDEO_PATH}"

model = YOLO(MODEL_PATH)
names = model.names

cap = cv2.VideoCapture(VIDEO_PATH)
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps= cap.get(cv2.CAP_PROP_FPS) or 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # thử 'avc1' nếu khó phát
writer = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (w, h))

prev = time.time()
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Dự đoán cho frame hiện tại
    preds = model.predict(
        frame, imgsz=img_size, conf=conf_thres, iou=iou_thres,
        device=device_prefer, verbose=False
    )

    for r in preds:
        boxes = r.boxes
        if boxes is None:
            continue
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cls_id = int(box.cls[0].item())
            conf   = float(box.conf[0].item())
            label  = f"{names.get(cls_id, cls_id)} {conf:.2f}"
            x1, y1, x2, y2 = map(int, [x1,y1,x2,y2])
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, label, (x1, y1-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # Hiển thị FPS
    now = time.time()
    fps_text = f"FPS: {1/(now-prev):.2f}"
    prev = now
    cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)

    writer.write(frame)
    frame_idx += 1

cap.release()
writer.release()
print("✅ Đã lưu:", OUTPUT_PATH)

# Hiển thị video kết quả
mp4 = open(OUTPUT_PATH,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=800 controls>
      <source src='{data_url}' type='video/mp4'>
</video>
""")

### Ghi chú & xử lý lỗi nhanh
- `FileNotFoundError` → kiểm tra lại `MODEL_PATH` và/hoặc `VIDEO_PATH`.
- `CUDA not available` → đặt `device_prefer = 'cpu'`.
- Chạy chậm → giảm `img_size` (vd 512), tăng `conf_thres`, đảm bảo dùng GPU.
- Video lỗi codec/không phát → đổi `fourcc` sang `'avc1'`.
- Tên lớp (class names) lấy từ model Ultralytics nên không cần tự nạp thủ công.